In [2]:
import pandas as pd

## Data

In [3]:
df = pd.DataFrame([
    {"week": "A", "day": "1_monday", "sales": 10},
    {"week": "A", "day": "5_friday", "sales": 8},
    {"week": "B", "day": "1_monday", "sales": 15},
    {"week": "B", "day": "5_friday", "sales": 7},
    {"week": "C", "day": "1_monday", "sales": 9},
    {"week": "C", "day": "5_friday", "sales": 3},
])


df[["week", "day", "sales"]]

,week,day,sales
0,A,1_monday,10
1,A,5_friday,8
2,B,1_monday,15
3,B,5_friday,7
4,C,1_monday,9
5,C,5_friday,3


## Relative frequency (percentage) within each group

This is the function I use most. Many times I use groupby to summarize some values and I want to know what is the percentage of the values in each group, not in all the data.
If we calculate the percentage directly in the summarized dataframe, the results will be calculated using all the data:

In [4]:
groupped_data = df.groupby(['week', 'day']).agg({'sales': 'sum'})
groupped_data["%"] = groupped_data.apply(lambda x:  100 * x / x.sum())
groupped_data

sales          %
week day                       
A    1_monday     10  19.230769
     5_friday      8  15.384615
B    1_monday     15  28.846154
     5_friday      7  13.461538
C    1_monday      9  17.307692
     5_friday      3   5.769231

For example, the percentage of the first row (19.23%) is dividing 10 by 52 (the sum of all sales). To calculate the percentage related to each week, we have to use groupby(level = 0):

In [5]:
groupped_data["%"] = groupped_data.groupby(level=0).apply(lambda x:  100 * x / x.sum())
groupped_data

sales          %
week day                       
A    1_monday     10  55.555556
     5_friday      8  44.444444
B    1_monday     15  68.181818
     5_friday      7  31.818182
C    1_monday      9  75.000000
     5_friday      3  25.000000

Now the percentage in the first row (55.55%) is comparing only the sales of the week A.

## Cumulative Sum

Many times I would like to perform a cumulative sum, in our example, to verify how many sales we perform until each day. To achieve this we need to use the cumsum() function:

In [6]:
groupped_data = df.groupby(['week', 'day']).agg({'sales': 'sum'})
groupped_data.cumsum()

sales
week day            
A    1_monday     10
     5_friday     18
B    1_monday     33
     5_friday     40
C    1_monday     49
     5_friday     52

To calculate the cumsum() in each week you can use the same trick of the last part (group level=0):


In [7]:
groupped_data.groupby(level=0).cumsum()

sales
week day            
A    1_monday     10
     5_friday     18
B    1_monday     15
     5_friday     22
C    1_monday      9
     5_friday     12

# Change the name of an aggregated metric

If you want to change the column name of an aggregated metric in the moment of the aggregation, you just need to do pass a tuple with the new column name and the aggregation function:

In [8]:
groupped_data = df.groupby("week").agg({"sales": [("max_sales","max"), ("mean_sales","mean")]})
groupped_data

sales           
     max_sales mean_sales
week                     
A           10          9
B           15         11
C            9          6

This could be useful if you want to remove the level "sales":

In [10]:
groupped_data.columns = groupped_data.columns.droplevel()
groupped_data

,max_sales,mean_sales
week,,
A,10,9
B,15,11
C,9,6


## Put the values into a list


If you want to put all the values of a group into a list, you just need to do this:

In [11]:
pd.DataFrame(df.groupby('week')['sales'].apply(list))

,sales
week,
A,"[10, 8]"
B,"[15, 7]"
C,"[9, 3]"


## Sources

- https://pandas.pydata.org/pandas-docs/stable/user_guide/groupby.html
- https://stackoverflow.com/questions/23377108/pandas-percentage-of-total-with-groupby
- https://stackoverflow.com/questions/22219004/grouping-rows-in-list-in-pandas-groupby

